In [2]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.linear_model import LogisticRegression

sys.path.append(r'C:\Users\DerDo\Desktop\fantasy_basketball_project')
from functions import *

In [3]:
### Get team data
okc = get_roster('OKC', '2026')
hou = get_roster('HOU', '2026')

### Get conference data
conference_data = get_team_data('2025')
east_df = conference_data[1]
west_df = conference_data[0]

east_df.columns =[col.replace('/', '') if '/' in col else col for col in east_df.columns]
east_df = east_df.rename(columns={'Eastern Conference': 'team_name', 'WL%':'WLperc'})
west_df.columns = [col.replace('/', '') if '/' in col else col for col in west_df.columns]
west_df = west_df.rename(columns={'Eastern Conference': 'team_name', 'WL%':'WLperc'})

east_df['season'] = '2025'
east_df['conference'] = 'east'
west_df['season'] = '2025'
west_df['conference'] = 'west'


creating tables and appending data

In [7]:
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE if not exists conference_data (
    team_name TEXT,
    W TEXT,
    L TEXT,
    WLperc TEXT,
    GB TEXT,
    PSG TEXT,
    PAG TEXT,
    SRS TEXT,
    season text,
    conference text,
    PRIMARY KEY (team_name, season)
);
""")

conn.commit()
conn.close()

# Connect to SQLite DB (creates table if not exists)
with sqlite3.connect("nba.db") as conn:
    # Write DataFrame to a table called "players"
    east_df.to_sql("conference_data", conn, if_exists="append", index=False)
    west_df.to_sql("conference_data", conn, if_exists="append", index=False)

    # Verify by reading back
    check = pd.read_sql("SELECT * FROM conference_data", conn)

In [9]:
check.head()

,team_name,W,L,WLperc,GB,PSG,PAG,SRS,season,conference
0,Cleveland Cavaliers*,64,18,.780,—,121.9,112.4,8.81,2025,east
1,Boston Celtics*,61,21,.744,3.0,116.3,107.2,8.28,2025,east
2,New York Knicks*,51,31,.622,13.0,115.8,111.7,3.59,2025,east
3,Indiana Pacers*,50,32,.610,14.0,117.4,115.1,1.68,2025,east
4,Milwaukee Bucks*,48,34,.585,16.0,115.5,113.0,2.12,2025,east


checking tables

In [8]:
with sqlite3.connect("nba.db") as conn:
    cur = conn.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cur.fetchall()

print("Tables:", [t[0] for t in tables])

Tables: ['conference_data']


dropping tables

In [5]:
with sqlite3.connect("nba.db") as conn:
    cur = conn.cursor()
    # cur.execute('drop table conference_data')
    cur.execute('drop table players')
    